# An Adaptive Partition-Based Approach for Solving Two-Stage Stochastic Programs with Fixed Recourse 
[Yongjia Song](mailto:yongjis@clemson.edu) and [James Luedtke](mailto:jrluedt1@wisc.edu)

In [1]:
# techincal lines
using LinearAlgebra
using JuMP
using Gurobi

#create gurobi environment
const GRB_ENV = Gurobi.Env();

Academic license - for non-commercial use only - expires 2021-05-08


In [2]:
#reading the data file
include("data.jl");

In [3]:
#define the model
MVP = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GRB_ENV), "OutputFlag" => 0));

#define the variables
@variable(MVP,u[1:nbFirstVars]); #u is the 1st stage variable in the MVP
@variable(MVP,v[1:nbSecVars]);   #v is the 2nd stage variable in the MVP


#define the objective
@objective(MVP,Min,sum(objcoef[i]*u[i] for i=1:nbFirstVars)+sum(objcoef[nbFirstVars+j]*v[j] for j=1:nbSecVars));


#define the constraints

#the 1st satge variable is bigger/smaller than its lower/upper bound
for i=1:nbFirstVars
    @constraint(MVP, u[i] >=firstvarlb[i]);
    @constraint(MVP, u[i] <=firstvarub[i]);
end

#the 2nd satge variable is bigger/smaller than the expected value of its lower/upper bound
for i=1:nbSecVars
    @constraint(MVP, v[i] >= (sum(secondvarlb[k,i] for k=1:nbScens)/nbScens));
    @constraint(MVP, v[i] <= (sum(secondvarub[k,i] for k=1:nbScens)/nbScens));
end

#the lower/upper bound constraints related to the 1st satge variable 
for i=1:nbFirstRows
    @constraint(MVP, sum(u[firstconstrind[i][j]]*firstconstrcoef[i][j] for j=1:length(firstconstrcoef[i])) >=firstconstrlb[i])    
    @constraint(MVP, sum(u[firstconstrind[i][j]]*firstconstrcoef[i][j] for j=1:length(firstconstrcoef[i])) <=firstconstrub[i])    
end

#the bound constraints related to the 2nd satge variable 
for i=1:nbSecRows
    if secondconstrsense[i] == 0
        @constraint(MVP, sum(u[second1stconstrind[i][j]]*second1stconstrcoef[i][j] for j=1:length(second1stconstrcoef[i]))+sum(v[second2ndconstrind[i][j]]*second2ndconstrcoef[i][j] for j=1:length(second2ndconstrcoef[i])) == (sum(secondconstrbd[j,i] for j=1:nbScens)/nbScens))
    end
    if secondconstrsense[i] == 1
        @constraint(MVP, sum(u[second1stconstrind[i][j]]*second1stconstrcoef[i][j] for j=1:length(second1stconstrcoef[i]))+sum(v[second2ndconstrind[i][j]]*second2ndconstrcoef[i][j] for j=1:length(second2ndconstrcoef[i])) >= (sum(secondconstrbd[j,i] for j=1:nbScens)/nbScens))
    end
    if secondconstrsense[i] == -1
        @constraint(MVP, sum(u[second1stconstrind[i][j]]*second1stconstrcoef[i][j] for j=1:length(second1stconstrcoef[i]))+sum(v[second2ndconstrind[i][j]]*second2ndconstrcoef[i][j] for j=1:length(second2ndconstrcoef[i])) <= (sum(secondconstrbd[j,i] for j=1:nbScens)/nbScens))
    end
end

In [4]:
optimize!(MVP)
status_mvp = termination_status(MVP);
if status_mvp != MOI.OPTIMAL
    println("Models status is: ", status_mvp, " === Oops! :/")
else
    obj_mvp = objective_value(MVP);
    xval = value.(u);
    yval = value.(v)
    println("Finish solving! Status = ", status_mvp);
    println("Optimal objective value = ", obj_mvp);
    println("x optimal value = ", xval);
    println("y optimal value = ", yval);
end

Finish solving! Status = OPTIMAL
Optimal objective value = 218.01398122003295
x optimal value = [0.0, 4.382227853768909, 0.0, 2.143650429190988, 0.15390412269472595, 0.0, 0.00907823792055301, 0.0, 0.4076869778625408, 0.0, 0.010530876431487727, 0.8259507917591504, 0.0, 0.0, 0.0, 0.07554298065344937, 0.1124766053641693, 0.29335021017563373, 0.1896740640303116, 0.13928779971105198]
y optimal value = [0.3701034309592, 0.4134238912422419, 2.245219202035589, 0.0, 0.0, 0.27798970024234154, 1.9257370492861778, 1.9669669988627463, 8.706661809364638, 2.872718146711897, 0.8105021826000701, 0.0, 2.8396558031285073, 0.0, 0.27609095167001063, 0.0, 2.2041596495119946, 0.17142382332988335, 0.0, 3.0010287475706714, 2.7729648641220646, 2.816580851704152, 2.11346308726969, 0.0, 0.0, 0.0, 7.054344486774584, 0.0, 0.0, 0.0]


In [5]:
function MS(P,p)
    Master = [];
    Master = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GRB_ENV), "OutputFlag" => 0));
    @variable(Master,x[1:nbFirstVars]);
    @variable(Master,yy[1:p,1:nbSecVars]);
    @objective(Master,Min,sum(objcoef[i]*x[i] for i=1:nbFirstVars)+sum(sum(objcoef[nbFirstVars+j]*yy[k,j]*(length(P[k])/nbScens) for k=1:p) for j=1:nbSecVars));


    #lower/upper bound on the 1st stage variable 
    for i=1:nbFirstVars
        @constraint(Master, x[i] >=firstvarlb[i]);
        @constraint(Master, x[i] <=firstvarub[i]);
    end

    #the lower/upper bound constraints related to the 1st satge variable 
    for i=1:nbFirstRows
        @constraint(Master, sum(x[firstconstrind[i][j]]*firstconstrcoef[i][j] for j=1:length(firstconstrcoef[i])) >=firstconstrlb[i]);    
        @constraint(Master, sum(x[firstconstrind[i][j]]*firstconstrcoef[i][j] for j=1:length(firstconstrcoef[i])) <=firstconstrub[i]);    
    end

    constrlbM = Array{Any,2}(undef, p, nbSecVars);
    for k=1:p, i=1:nbSecVars
        constrlbM[k,i]=@constraint(Master, yy[k,i] >= sum(secondvarlb[j,i] for j in P[k])/length(P[k]));
    end
    
    construbM = Array{Any,2}(undef, p, nbSecVars);
    for k=1:p, i=1:nbSecVars
        construbM[k,i]=@constraint(Master, yy[k,i] <= sum(secondvarub[j,i] for j in P[k])/length(P[k]));
    end
    
    constrM = Array{Any,2}(undef, p, nbSecRows);
    for k=1:p
        for i=1:nbSecRows
            if secondconstrsense[i] == 0
                constrM[k,i]=@constraint(Master, sum(yy[k,second2ndconstrind[i][j]]*second2ndconstrcoef[i][j] for j=1:length(second2ndconstrcoef[i]))+sum(x[second1stconstrind[i][j]]*second1stconstrcoef[i][j] for j=1:length(second1stconstrcoef[i])) == sum(secondconstrbd[s,i] for s in P[k])/length(P[k]));
            end
            if secondconstrsense[i] == 1
                constrM[k,i]=@constraint(Master, sum(yy[k,second2ndconstrind[i][j]]*second2ndconstrcoef[i][j] for j=1:length(second2ndconstrcoef[i]))+sum(x[second1stconstrind[i][j]]*second1stconstrcoef[i][j] for j=1:length(second1stconstrcoef[i])) >= sum(secondconstrbd[s,i] for s in P[k])/length(P[k]));
            end
            if secondconstrsense[i] == -1
                constrM[k,i]=@constraint(Master, sum(yy[k,second2ndconstrind[i][j]]*second2ndconstrcoef[i][j] for j=1:length(second2ndconstrcoef[i]))+sum(x[second1stconstrind[i][j]]*second1stconstrcoef[i][j] for j=1:length(second1stconstrcoef[i])) <= sum(secondconstrbd[s,i] for s in P[k])/length(P[k]));
            end
        end
    end
    piM = zeros(p,nbSecRows+nbSecVars*2);
    
    optimize!(Master)
    status_master = termination_status(Master);
    if status_master != MOI.OPTIMAL
        println("Models status is: ", status_master, " === Oops! :/")
        exit()
    else
        LB = objective_value(Master);
        xval = value.(x);
        yval = value.(yy)
    end
    
    for j=1:p
        for i=1:nbSecVars
            piM[j,i] = dual(constrlbM[j,i]);
            piM[j,nbSecVars+i] = dual(construbM[j,i]);
        end
        for i=1:nbSecRows
            piM[j,nbSecVars+nbSecVars+i]= dual(constrM[j,i]);
        end        
    end
    return Master, status_master, xval, yval, LB, piM
end

MS (generic function with 1 method)

In [6]:
function SP(xval,UB)
    SubProblem = [];
    SubProblem = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GRB_ENV), "OutputFlag" => 0));
    @variable(SubProblem,y[1:nbSecVars]);
    @objective(SubProblem,Min,sum(objcoef[nbFirstVars+j]*y[j] for j=1:nbSecVars));

    #We will initilaize the RHS of all the 2nd stage constraints to be zero and we will change it later
    @constraint(SubProblem,constrlb[i=1:nbSecVars], y[i] >= 0);
    @constraint(SubProblem,construb[i=1:nbSecVars], y[i] <= 0);
    @constraint(SubProblem,constr1[i=1:nbSecRows; secondconstrsense[i] == 0], sum(y[second2ndconstrind[i][j]]*second2ndconstrcoef[i][j] for j=1:length(second2ndconstrcoef[i])) == 0);
    @constraint(SubProblem,constr2[i=1:nbSecRows; secondconstrsense[i] == 1], sum(y[second2ndconstrind[i][j]]*second2ndconstrcoef[i][j] for j=1:length(second2ndconstrcoef[i])) >= 0);
    @constraint(SubProblem,constr3[i=1:nbSecRows; secondconstrsense[i] == -1], sum(y[second2ndconstrind[i][j]]*second2ndconstrcoef[i][j] for j=1:length(second2ndconstrcoef[i])) <= 0);
    
    pi = zeros(nbScens,nbSecRows+nbSecVars*2);
    Q = zeros(nbScens);
    #For each scenario we will do the following 
    for k=1:nbScens
        for i=1:nbSecVars
            #update the 2nd stage variable LB & UB
            TempRHSlb = secondvarlb[k,i]
            set_normalized_rhs(constrlb[i], TempRHSlb);
            TempRHSub = secondvarub[k,i]
            set_normalized_rhs(construb[i], TempRHSub);
        end
        
        #update the bound of each of the constraints 
        for i=1:nbSecRows
            if secondconstrsense[i] == 0
                TempRHS1 = secondconstrbd[k,i] - sum(xval[second1stconstrind[i][j]]*second1stconstrcoef[i][j] for j=1:length(second1stconstrcoef[i]))
                set_normalized_rhs(constr1[i], TempRHS1);
                
            end
            if secondconstrsense[i] == 1
                TempRHS2 = secondconstrbd[k,i] - sum(xval[second1stconstrind[i][j]]*second1stconstrcoef[i][j] for j=1:length(second1stconstrcoef[i]))
                set_normalized_rhs(constr2[i], TempRHS2);
            end
            if secondconstrsense[i] == -1
                TempRHS3 = secondconstrbd[k,i] - sum(xval[second1stconstrind[i][j]]*second1stconstrcoef[i][j] for j=1:length(second1stconstrcoef[i]))
                set_normalized_rhs(constr3[i], TempRHS3);
            end
        end
        #We solve the subproblen & obtain its objective value
        optimize!(SubProblem)
        statusSP = termination_status(SubProblem);
        if statusSP != MOI.OPTIMAL
            println("Models status is: ", statusSP, " === Oops! :/")
            exit()
        else
            Q[k] = objective_value(SubProblem);
            #We get the dual multiplier of each constraint for this scenario 
            for i=1:nbSecVars
                pi[k,i] = dual(constrlb[i])
                pi[k,nbSecVars+i] = dual(construb[i])
            end
            for i=1:nbSecRows
                if secondconstrsense[i] == 0
                    pi[k,nbSecVars+nbSecVars+i]=dual(constr1[i]);
                end
                if secondconstrsense[i] == 1
                    pi[k,nbSecVars+nbSecVars+i]=dual(constr2[i]);
                end
                if secondconstrsense[i] == -1
                    pi[k,nbSecVars+nbSecVars+i]=dual(constr3[i]);
                end  
            end
            
        end

    end
    
    tempUB=sum(objcoef[i]*xval[i] for i=1:nbFirstVars)+(1/nbScens)*(sum(Q[i] for i=1:nbScens));
    
    #We check, if the new solution improves the upper bound or not, if yes; we update the UB, if not we keep the old UB
    if tempUB < UB
        UB = tempUB
    end
    return SubProblem, Q, UB, pi
end

SP (generic function with 1 method)

In [7]:
 function Refine(P,PC,g,pi,ϵ,xxval,Q)
    Refine_Start=time();
    temp = [];
    reps = [];
    reps_indc = [];
    push!(temp,[PC[g][1]])
    push!(reps,pi[PC[g][1],:]);
    push!(reps_indc,PC[g][1]);
    
    for k=2:length(PC[g])
        count = 0;
        for s=1:length(temp)
            diff = pi[PC[g][k],:]-reps[s];
            if norm(diff) < ϵ 
                push!(temp[s],PC[g][k])
                break
            else
                #check if the solution is degenerate solutions
                obj_diff = Q[PC[g][k]] - Q[reps_indc[s]]
                if abs(obj_diff)/abs(Q[reps_indc[s]]) < ϵ
                    push!(temp[s],PC[g][k])
                    break
                else
                    count +=1
                end
            end
        end
        if count == length(temp)
            push!(temp,[PC[g][k]])
            push!(reps, pi[PC[g][k],:]);
            push!(reps_indc, PC[g][k]);
        end
    end 
    
    filter!(e->e!=PC[g],P)
    for i=1:length(temp)
        push!(P,temp[i])
    end
    
    elaps = time()-Refine_Start;
    return P, elaps
end

Refine (generic function with 1 method)

In [8]:
#initialize stuff
P = [[k for k=1:nbScens]]; #The initial partition
PC = deepcopy(P); #a copy of the partition
p = length(P); 

LB = obj_mvp;
UB = 1e10;

ϵ = 1e-5;
iter = 0;

start=time();
while (UB-LB)*1.0/max(1e-10,abs(UB)) > ϵ
    iter = iter + 1;
    
    Master, statusMS, xval, yval, LB, piM = MS(P,p);
    SubProblem, Q, UB, pi = SP(xval,UB);
    
    #given the new dual multiplier, refine the partition
    #check to make sure # of clusters < the total # of scenarios
    println("iter = ", iter);
    println("LB = ", LB);
    println("UB = ", UB);
    println("===============================================================");
    
    if length(PC) < nbScens
        for c=1:length(PC)
            if length(PC[c]) > 1
                P, elaps = Refine(P,PC,c,pi,ϵ,xval,Q);
            end
        end
        PC = deepcopy(P);
        p = length(P); 
    end
    
end
Elapsed = time()-start;
println("******************************************")
println("******************************************")
println("Problem is solved! Yaay")
println("******************************************")
println("Elapsed time = ", Elapsed, " seconds")
println("iter = ", iter);
println("# of clusters used  = ", p);
println("LB = ", LB);
println("UB = ", UB);
println("xvalue = ", xval);

iter = 1
LB = 218.01398122003295
UB = 272.48302690625263
iter = 2
LB = 237.28274353830196
UB = 250.87842256316026
iter = 3
LB = 248.66347645099043
UB = 250.87842256316026
iter = 4
LB = 249.9074023718201
UB = 250.71910494025713
iter = 5
LB = 250.4240696159045
UB = 250.59053961426895
iter = 6
LB = 250.55385918565995
UB = 250.59053961426895
iter = 7
LB = 250.57219573969408
UB = 250.5805665495743
iter = 8
LB = 250.57981836312166
UB = 250.5805665495743
******************************************
******************************************
Problem is solved! Yaay
******************************************
Elapsed time = 39.554747104644775 seconds
iter = 8
# of clusters used  = 693
LB = 250.57981836312166
UB = 250.5805665495743
xvalue = [0.0, 3.1133036811595796, 0.0, 1.471386430807964, 0.24922915256672745, 0.0, 0.030380150643928173, 0.013645430356848825, 0.40640488902330413, 0.0, 0.0, 0.6847947780949112, 0.0, 0.0, 0.0, 0.08129155361359267, 0.11678827840943277, 0.2856350201501783, 0.191544760043